In [1]:
import os

In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Root007x/data_science_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Root007x"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f609507313312db73d17bae15646bf188d6f93e5"

In [3]:
%pwd

'c:\\Users\\ROOT\\Desktop\\mlops_projects\\data_science_project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\ROOT\\Desktop\\mlops_projects\\data_science_project'

In [6]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 3C50-BC77

 Directory of c:\Users\ROOT\Desktop\mlops_projects\data_science_project

05/20/2025  09:21 AM    <DIR>          .
05/20/2025  09:21 AM    <DIR>          ..
05/20/2025  06:33 AM    <DIR>          .github
05/20/2025  09:00 AM                24 .gitignore
05/21/2025  03:17 AM    <DIR>          artifacts
05/20/2025  06:33 AM    <DIR>          config
05/20/2025  06:36 AM                 0 DockerFile
05/20/2025  06:02 AM            35,823 LICENSE
05/20/2025  08:25 AM    <DIR>          logs
05/21/2025  02:37 AM             1,854 main.py
05/21/2025  01:12 AM                42 params.yaml
05/21/2025  12:17 AM               399 README.md
05/20/2025  06:11 AM               132 requirements.txt
05/21/2025  02:43 AM    <DIR>          research
05/20/2025  04:19 PM               333 schema.yaml
05/20/2025  06:36 AM                 0 setup.py
05/20/2025  06:33 AM    <DIR>          src
05/20/2025  09:07 AM             1,452 template.p

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: str
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(self,
                config_file_path=CONFIG_FILE_PATH,
                params_file_path=PARAMS_FILE_PATH,
                schema_file_path=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri= os.environ.get("MLFLOW_TRACKING_URI")

        )
        return model_evaluation_config

In [10]:
import os
import pandas as pd 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np 
import joblib

In [13]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_X = test_data.drop(columns=[self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Start
        with mlflow.start_run():
            predicted_qualities = model.predict(test_X)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            # Save result as json format
            save_json(Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model,"model")

In [14]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config=model_eval_config)
    model_eval.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-21 03:36:47,066: INFO: common: Yaml File: config\config.yaml loaded successfully]
[2025-05-21 03:36:47,069: INFO: common: Yaml File: params.yaml loaded successfully]
[2025-05-21 03:36:47,073: INFO: common: Yaml File: schema.yaml loaded successfully]
[2025-05-21 03:36:47,074: INFO: common: Created Directory at: artifacts]
[2025-05-21 03:36:47,076: INFO: common: Created Directory at: artifacts/model_evaluation]
[2025-05-21 03:36:47,442: INFO: common: Json File saved at: artifacts\model_evaluation\metrics.json]


2025/05/21 03:36:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2025/05/21 03:37:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run illustrious-rat-227 at: https://dagshub.com/Root007x/data_science_project.mlflow/#/experiments/0/runs/fb17dbd543e24609a43ca21eaab87704
🧪 View experiment at: https://dagshub.com/Root007x/data_science_project.mlflow/#/experiments/0
